# Machine learning - Sistema de recomendación

In [33]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pyarrow as pa
import pyarrow.parquet as pq

### Preprocesamiento de los datos.

In [34]:
df_juegos = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\PI-MLOps\\Datasets\\Procesado\\juegos.parquet')
df_resenias = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\PI-MLOps\\Datasets\\Procesado\\resenias.parquet')

In [35]:
df_resenias.sample()

,user_id,item_id,recommend,analisis_sentimiento
46103,76561198038567931,219150,True,2


In [36]:
df_juegos.sample()

,genres,title,price,id,developer,release_year
3097,action,bioshock infinite: burial at sea - episode two,14.99,214932,"irrational games,aspyr (mac),virtual programmi...",2014


In [37]:
# Se cambia el nombre de la columna "id" por "item_id" para poder realizar el merge de los dataframes
df_juegos.rename(columns={'id': 'item_id'}, inplace=True)
df_juegos.sample()

,genres,title,price,item_id,developer,release_year
4747,adventure,pop: methodology experiment one - game of the ...,0.99,316070,rob lach,2014


In [38]:
# Se extraen las columnas necesarias de sus respectivos dataframes.
df_jue_ml = df_juegos[['item_id', 'genres', 'title', 'release_year', 'developer']]
df_res_ml = df_resenias[['item_id','recommend', 'analisis_sentimiento']]
df_ml = df_res_ml.merge(df_jue_ml, on='item_id', how='inner')

In [39]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120671 entries, 0 to 120670
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   item_id               120671 non-null  object
 1   recommend             120671 non-null  bool  
 2   analisis_sentimiento  120671 non-null  int64 
 3   genres                120671 non-null  object
 4   title                 120671 non-null  object
 5   release_year          120671 non-null  int32 
 6   developer             120671 non-null  object
dtypes: bool(1), int32(1), int64(1), object(4)
memory usage: 5.2+ MB


In [40]:
# Se crea una nueva columna combinando las columnas existentes "genres", "title" y "developer".
df_ml['combined'] = df_ml['genres'] + ' ' + df_ml['title'] + ' ' + df_ml['developer']

# Se eliminan las columnas "genres" y "developer"
df_ml = df_ml.drop(['genres', 'developer'], axis=1)

In [41]:
df_ml['item_id'] = df_ml['item_id'].fillna(0)
# Se eliminan las filas con valores igual a cero em la columna "item_id" ya que no corresponden a un número id.
df_ml = df_ml[df_ml['item_id'] != 0]
df_ml['item_id'] = df_ml['item_id'].astype(np.int32)
df_ml['analisis_sentimiento'] = df_ml['analisis_sentimiento'].fillna(0)
df_ml['analisis_sentimiento'] = df_ml['analisis_sentimiento'].astype(np.int16)
df_ml['release_year'] = df_ml['release_year'].fillna(0)
# Se eliminan las filas que tienen valores iguales a cero en la columna "playtime_forever" ya que no se trata de años.
df_ml = df_ml[df_ml['release_year'] != 0]
df_ml['release_year'] = df_ml['release_year'].astype(np.int16)

# Se eliminan los datos duplicados.
df_ml = df_ml.drop_duplicates()

#Se toma una muestra del dataframe.
df_ml = df_ml.sample(n=3000, random_state=42, replace=True)

# Se hace un reset del índice.
df_ml.reset_index(inplace=True, drop=True)

In [42]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_id               3000 non-null   int32 
 1   recommend             3000 non-null   bool  
 2   analisis_sentimiento  3000 non-null   int16 
 3   title                 3000 non-null   object
 4   release_year          3000 non-null   int16 
 5   combined              3000 non-null   object
dtypes: bool(1), int16(2), int32(1), object(2)
memory usage: 73.4+ KB


In [43]:
df_ml

,item_id,recommend,analisis_sentimiento,title,release_year,combined
0,274170,True,1,hotline miami 2: wrong number,2015,indie hotline miami 2: wrong number dennaton g...
1,380360,True,0,tap tap infinity,2015,casual tap tap infinity scary bee llc
2,417860,False,0,emily is away,2015,indie emily is away kyle seeley
3,216150,True,2,maplestory,2012,rpg maplestory nexon
4,475490,True,1,major\minor - complete edition,2016,adventure major\minor - complete edition klace
...,...,...,...,...,...,...
2995,287220,False,1,autocraft,2017,indie autocraft alientrap
2996,231740,True,1,knights of pen and paper +1 edition,2013,indie knights of pen and paper +1 edition beho...
2997,298340,True,2,flashout 2,2014,action flashout 2 jujubee s.a.
2998,261980,True,2,half-life: before,2009,indie half-life: before andrii vintsevych


In [57]:
def recomendacion_juego(item_id):

    # Se crea una instancia de TfidfVectorizer y aplica fit_transform.
    vectorizador = TfidfVectorizer()
    matriz = vectorizador.fit_transform(df_ml['combined'])

    # Se calcula la similitud del coseno.
    coseno = cosine_similarity(matriz, matriz)

    # Se verifica si el item_id existe en el DataFrame.
    if item_id not in df_ml['item_id'].values:
        return "Error: item_id no encontrado."
    
    # Se busca el índice correspondiente al item_id.
    item_indice = df_ml[df_ml['item_id'] == item_id].index[0]
    
    # Se consiguen las puntuaciones de similitud para ese ítem con todos los demás ítems.
    puntuacion_similar = list(enumerate(coseno[item_indice]))
    
    # Se ordenan los ítems por puntuaciones de similitud en orden descendente.
    puntuacion_similar = sorted(puntuacion_similar, key=lambda x: x[1], reverse=True)
    
    # Se obtienen los índices de los ítems con mayor simlitud (excluyendo el ítem en sí).
    puntuacion_similar = puntuacion_similar[1:6]
    
    # Se obtienen los índices de los ítems más similares.
    item_indices = [i[0] for i in puntuacion_similar]
    
    # Se obtiene el título del ítem ingresado.
    item_title = df_ml.loc[df_ml['item_id'] == item_id, 'title'].values[0]
    
    # Se obtienen los títulos de los ítems más similares y se los convierte a formato title.
    titulos = [titulo.title() for titulo in df_ml.iloc[item_indices]['title'].tolist()]
    
    return f"Los títulos similares a '{item_title.title()}' son: {titulos}"

In [59]:
recomendacion_juego(335660)

"Los títulos similares a 'Cafe 0 ~The Drowned Mermaid~' son: ['The Stanley Parable', 'The Stanley Parable', 'Cradle', 'The Night Of The Rabbit', 'The Quest']"

In [46]:
recomendacion_juego(380360)

"Los títulos similares a 'Tap Tap Infinity' son: ['Call Of Duty® 2', 'Gems Of War - Puzzle Rpg', 'Call Of Duty®: Ghosts - Squad Pack - Resistance', 'Turnon', 'Cubic Castles']"

### Carga de datos

In [24]:
table = pa.Table.from_pandas(df_ml)
pq.write_table(table, 'df_ml.parquet')